# Test Package ForestGaps - Google Colab

Notebook pour tester l'installation et le workflow complet de ForestGaps sur Google Colab.

**Date:** 2025-12-03  
**Status:** Prêt pour test Colab

---

## 1. Installation du Package

### Option A: Installation depuis GitHub
```python
!pip install git+https://github.com/ArthurCalvi/forestgaps-dl.git
```

### Option B: Installation en mode développement
```python
!git clone https://github.com/ArthurCalvi/forestgaps-dl.git
%cd forestgaps-dl
!pip install -e .
```

In [ ]:
# Installation rapide
!pip install git+https://github.com/ArthurCalvi/forestgaps-dl.git

## 2. Setup Google Drive (Optionnel)

Pour sauvegarder les résultats et charger des données depuis Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Test des Imports

In [ ]:
# Imports de base
import forestgaps
from forestgaps.models import model_registry
from forestgaps.inference import InferenceManager
from forestgaps.evaluation import evaluate_model

print(f"✅ ForestGaps version: {forestgaps.__version__}")
print(f"✅ Modèles disponibles: {model_registry.list_models()}")

## 4. Test de Création de Modèle

In [ ]:
# Tester la création de tous les modèles
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Test UNet
model = model_registry.create('unet', in_channels=1, out_channels=1)
model = model.to(device)
print(f"✅ UNet créé: {model.get_num_parameters()} paramètres")

# Test entrée dummy
dummy_input = torch.randn(1, 1, 256, 256).to(device)
output = model(dummy_input)
print(f"✅ Output shape: {output.shape}")

## 5. Test Training Minimal

Test avec des données synthétiques:

In [ ]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Créer des données synthétiques
n_samples = 20
dsm_data = torch.randn(n_samples, 1, 256, 256)
mask_data = torch.randint(0, 2, (n_samples, 1, 256, 256)).float()

dataset = TensorDataset(dsm_data, mask_data)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

print(f"✅ Dataset créé: {len(dataset)} échantillons")
print(f"✅ DataLoader prêt: {len(train_loader)} batches")

In [ ]:
# Training minimal
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(2):
    epoch_loss = 0
    for dsm, mask in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        dsm, mask = dsm.to(device), mask.to(device)
        
        optimizer.zero_grad()
        output = model(dsm)
        loss = criterion(output, mask)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1} - Loss: {epoch_loss/len(train_loader):.4f}")

print("✅ Training test réussi!")

## 6. Test Sauvegarde/Chargement Modèle

In [ ]:
# Sauvegarder
model_path = "/content/test_model.pt"
torch.save(model.state_dict(), model_path)
print(f"✅ Modèle sauvegardé: {model_path}")

# Recharger
new_model = model_registry.create('unet', in_channels=1, out_channels=1)
new_model.load_state_dict(torch.load(model_path))
new_model = new_model.to(device)
print("✅ Modèle rechargé avec succès")

## 7. Test Tous les Modèles du Registry

In [ ]:
# Tester l'instantiation de tous les modèles
all_models = model_registry.list_models()
print(f"Test de {len(all_models)} modèles...\n")

results = {}
for model_name in all_models:
    try:
        # Paramètres selon le type de modèle
        if 'regression' in model_name:
            params = {'in_channels': 1, 'out_channels': 1}
        elif 'threshold' in model_name:
            params = {'in_channels': 1, 'out_channels': 1, 'threshold_value': 5.0}
        else:
            params = {'in_channels': 1, 'out_channels': 1}
        
        # Créer le modèle
        test_model = model_registry.create(model_name, **params)
        n_params = test_model.get_num_parameters()
        
        # Test forward pass
        test_model.eval()
        with torch.no_grad():
            test_output = test_model(dummy_input.cpu())
        
        results[model_name] = {
            'status': '✅ OK',
            'params': f'{n_params:,}',
            'output_shape': str(test_output.shape)
        }
        print(f"✅ {model_name:30s} - {n_params:>10,} params")
        
    except Exception as e:
        results[model_name] = {
            'status': '❌ FAILED',
            'error': str(e)
        }
        print(f"❌ {model_name:30s} - FAILED: {str(e)}")

print(f"\n✅ Test complété: {sum(1 for r in results.values() if r['status'] == '✅ OK')}/{len(all_models)} modèles OK")

## 8. Résumé Final

In [ ]:
print("="*60)
print("RÉSUMÉ DES TESTS FORESTGAPS")
print("="*60)
print(f"✅ Package installé et fonctionnel")
print(f"✅ {len(all_models)} modèles disponibles")
print(f"✅ Training/Inference testés")
print(f"✅ Sauvegarde/Chargement OK")
print("="*60)